# Quantum Encoding Methods for Earth Observation Data

## Introduction to Quantum Machine Learning for Remote Sensing

In this notebook, we'll explore different methods of encoding classical Earth observation (EO) data into quantum states. This is a crucial first step in quantum machine learning, as quantum computers operate on quantum states rather than classical bits.

**Why Quantum Encoding Matters for Earth Observation:**
- EO datasets are often high-dimensional (multispectral/hyperspectral imagery)
- Quantum states can represent exponentially large feature spaces
- Different encoding strategies affect which features are emphasized
- Choice of encoding impacts the performance of quantum machine learning models

We'll use the **EuroSAT dataset**, which contains Sentinel-2 satellite images classified into 10 land use categories.

## Setup and Dependencies

First, let's install and import the necessary libraries:

In [ ]:
# Install required packages
!pip install pennylane scikit-learn -q

In [ ]:
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import zipfile
import urllib.request
from pathlib import Path

# Set random seed for reproducibility
np.random.seed(42)

print(f"PennyLane version: {qml.__version__}")
print(f"NumPy version: {np.__version__}")

## Download and Prepare EuroSAT Dataset

The EuroSAT dataset contains RGB images (64×64 pixels) from Sentinel-2 satellites, covering 10 land use classes:
- Annual Crop, Forest, Herbaceous Vegetation, Highway, Industrial, Pasture, Permanent Crop, Residential, River, Sea/Lake

In [ ]:
import ssl
import urllib.request

def download_and_extract_eurosat(data_dir="./eurosat_data"):
    """Download and extract the EuroSAT dataset"""

    url = "https://madm.dfki.de/files/sentinel/EuroSAT.zip"
    zip_path = os.path.join(data_dir, "EuroSAT.zip")

    # Create directory if it doesn't exist
    os.makedirs(data_dir, exist_ok=True)

    # Download if not already present
    if not os.path.exists(zip_path):
        print(f"Downloading EuroSAT dataset from {url}...")

        # Create an SSL context that doesn't verify certificates
        ssl_context = ssl.create_default_context()
        ssl_context.check_hostname = False
        ssl_context.verify_mode = ssl.CERT_NONE

        # Download with unverified SSL
        with urllib.request.urlopen(url, context=ssl_context) as response:
            with open(zip_path, 'wb') as out_file:
                out_file.write(response.read())

        print("Download complete!")
    else:
        print("Dataset already downloaded.")

    # Extract if not already extracted
    extract_path = os.path.join(data_dir, "2750")
    if not os.path.exists(extract_path):
        print("Extracting dataset...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(data_dir)
        print("Extraction complete!")
    else:
        print("Dataset already extracted.")

    return extract_path

# Download and extract the dataset
dataset_path = download_and_extract_eurosat()

## Load and Visualize Sample EO Data

In [ ]:
def load_sample_images(dataset_path, n_samples=5):
    """Load sample images from different classes"""

    classes = sorted([d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))])
    samples = []

    for cls in classes[:n_samples]:
        class_path = os.path.join(dataset_path, cls)
        images = [f for f in os.listdir(class_path) if f.endswith('.jpg')]
        if images:
            img_path = os.path.join(class_path, images[0])
            img = Image.open(img_path)
            samples.append((cls, np.array(img)))

    return samples

# Load samples
samples = load_sample_images(dataset_path)

# Visualize
fig, axes = plt.subplots(1, len(samples), figsize=(15, 3))
for idx, (class_name, img) in enumerate(samples):
    axes[idx].imshow(img)
    axes[idx].set_title(class_name)
    axes[idx].axis('off')
plt.tight_layout()
plt.show()

## Preprocessing: From Satellite Image to Feature Vector

Before we can encode data into quantum states, we need to preprocess our images into normalized feature vectors.

In [ ]:
def preprocess_image(img, target_size=8):
    """
    Preprocess satellite image for quantum encoding

    Args:
        img: Input image (H x W x C)
        target_size: Number of features to extract

    Returns:
        Normalized feature vector
    """
    # Convert to grayscale if RGB
    if len(img.shape) == 3:
        img_gray = np.mean(img, axis=2)
    else:
        img_gray = img

    # Resize to small patch and flatten
    from PIL import Image as PILImage
    img_resized = PILImage.fromarray(img_gray.astype(np.uint8)).resize((int(np.sqrt(target_size)),
                                                                          int(np.sqrt(target_size))))
    features = np.array(img_resized).flatten()

    # Normalize to [0, 1]
    features = features / 255.0

    # Further normalize to have unit norm for some encoding schemes
    features = features / (np.linalg.norm(features) + 1e-10)

    return features

# Example: preprocess one image
sample_img = samples[0][1]
features = preprocess_image(sample_img, target_size=8)

print(f"Original image shape: {sample_img.shape}")
print(f"Feature vector shape: {features.shape}")
print(f"Feature vector (first 8 values): {features}")
print(f"Feature vector norm: {np.linalg.norm(features):.4f}")

# Part 1: Basis Encoding (Digital Encoding)

## Theory

**Basis encoding** (also called digital encoding) is the most straightforward encoding method. It directly maps classical binary data to the computational basis states of qubits.

### Mathematical Formulation

For a classical bit string $x = x_1 x_2 ... x_n$ where $x_i \in \{0, 1\}$, basis encoding creates:

$$|x\rangle = |x_1\rangle \otimes |x_2\rangle \otimes ... \otimes |x_n\rangle$$

### Properties:
- **Direct mapping**: Each classical bit → one qubit
- **No superposition**: States are computational basis states
- **Information capacity**: $n$ qubits encode $n$ classical bits
- **Use case**: Good for discrete/categorical features

### For Earth Observation:
In EO applications, we might use basis encoding for:
- Land cover classification labels (binary or one-hot encoded)
- Thresholded pixel values (cloud/no-cloud, water/land)
- Discretized spectral indices (NDVI ranges)

In [ ]:
def basis_encoding(features, threshold=0.5):
    """
    Encode features using basis encoding

    Args:
        features: Normalized feature vector [0, 1]
        threshold: Threshold for binarization

    Returns:
        Binary string representation
    """
    # Binarize features
    binary_features = (features > threshold).astype(int)
    return binary_features

# Apply basis encoding
n_qubits = 8
# Make sure we use exactly n_qubits features
features_subset = features[:n_qubits]
binary_encoded = basis_encoding(features_subset)

print("Basis Encoding Example:")
print(f"Original features: {features_subset}")
print(f"Binary encoded:    {binary_encoded}")
print(f"Quantum state:     |{''.join(map(str, binary_encoded))}⟩")

# Verify shapes match
print(f"\nShape verification:")
print(f"features_subset shape: {features_subset.shape}")
print(f"binary_encoded shape: {binary_encoded.shape}")

### Implementing Basis Encoding in PennyLane

In [ ]:
# Define quantum device
dev_basis = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev_basis)
def basis_encoding_circuit(binary_data):
    """
    Create a quantum circuit with basis encoding

    Args:
        binary_data: Binary feature vector
    """
    # Apply X gates where bit is 1
    for i, bit in enumerate(binary_data):
        if bit == 1:
            qml.PauliX(wires=i)

    # Return state vector
    return qml.state()

# Execute circuit
quantum_state_basis = basis_encoding_circuit(binary_encoded)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

# Classical data - ensure both arrays have the same length
ax1.bar(range(len(features_subset)), features_subset, color='blue', alpha=0.6, label='Original')
ax1.bar(range(len(binary_encoded)), binary_encoded, color='red', alpha=0.6, label='Binarized')
ax1.set_xlabel('Feature Index')
ax1.set_ylabel('Value')
ax1.set_title('Classical Feature → Binary Encoding')
ax1.legend()
ax1.set_xticks(range(len(features_subset)))

# Quantum state amplitudes
basis_states = [f"|{i:0{n_qubits}b}⟩" for i in range(2**n_qubits)]
amplitudes = np.abs(quantum_state_basis)**2

# Only show non-zero amplitude
non_zero_idx = np.where(amplitudes > 1e-10)[0]
ax2.bar(non_zero_idx, amplitudes[non_zero_idx], color='green')
ax2.set_xlabel('Basis State')
ax2.set_ylabel('Probability')
ax2.set_title(f'Quantum State: {basis_states[non_zero_idx[0]]}')
ax2.set_xticks(non_zero_idx)
ax2.set_xticklabels([basis_states[i] for i in non_zero_idx], rotation=45)

plt.tight_layout()
plt.show()

print(f"\nThe encoded state is a single basis state: {basis_states[non_zero_idx[0]]}")
print(f"Probability of measuring this state: {amplitudes[non_zero_idx[0]]:.4f}")

# Part 2: Amplitude Encoding (Dense Encoding)

## Theory

**Amplitude encoding** embeds classical data into the amplitudes of a quantum state, providing an exponential advantage in data representation.

### Mathematical Formulation

For a classical vector $\mathbf{x} = (x_1, x_2, ..., x_N)$ where $N = 2^n$, amplitude encoding creates:

$$|\psi\rangle = \sum_{i=1}^{N} x_i |i\rangle$$

where the state must be normalized: $\sum_{i=1}^{N} |x_i|^2 = 1$

### Properties:
- **Exponential capacity**: $n$ qubits encode $2^n$ classical values
- **Superposition**: Data encoded in quantum amplitudes
- **Normalization required**: Features must form valid quantum state
- **Use case**: Optimal for high-dimensional continuous data

### For Earth Observation:
Perfect for encoding:
- Normalized pixel intensities across spectral bands
- Spatial feature vectors from image patches
- Compressed representations of hyperspectral signatures

**Challenge**: Requires $2^n$ amplitudes, so we need $\text{log}_2(\text{features})$ qubits

In [ ]:
def amplitude_encoding_prep(features):
    """
    Prepare features for amplitude encoding

    Args:
        features: Feature vector

    Returns:
        Normalized feature vector padded to power of 2
    """
    # Pad to nearest power of 2
    n_features = len(features)
    n_qubits = int(np.ceil(np.log2(n_features)))
    n_amplitudes = 2 ** n_qubits

    # Pad with zeros
    padded = np.zeros(n_amplitudes)
    padded[:n_features] = features

    # Normalize
    norm = np.linalg.norm(padded)
    if norm > 0:
        padded = padded / norm

    return padded, n_qubits

# Prepare data
amplitudes, n_qubits_amp = amplitude_encoding_prep(features)

print(f"Original features: {len(features)} values")
print(f"Required qubits: {n_qubits_amp}")
print(f"Amplitude vector: {len(amplitudes)} values (2^{n_qubits_amp})")
print(f"First 8 amplitudes: {amplitudes[:8]}")
print(f"Norm check: {np.linalg.norm(amplitudes):.6f} (should be 1.0)")

### Implementing Amplitude Encoding in PennyLane

In [ ]:
dev_amplitude = qml.device('default.qubit', wires=n_qubits_amp)

@qml.qnode(dev_amplitude)
def amplitude_encoding_circuit(amplitudes):
    """
    Create quantum circuit with amplitude encoding

    Args:
        amplitudes: Normalized amplitude vector
    """
    # PennyLane's built-in amplitude embedding
    qml.AmplitudeEmbedding(features=amplitudes, wires=range(n_qubits_amp), normalize=True)

    return qml.state()

# Execute circuit
quantum_state_amplitude = amplitude_encoding_circuit(amplitudes)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

# Classical amplitudes
ax1.bar(range(len(amplitudes)), amplitudes, color='blue', alpha=0.7)
ax1.set_xlabel('Index')
ax1.set_ylabel('Amplitude')
ax1.set_title('Classical Data (Normalized)')
ax1.axhline(y=0, color='k', linestyle='-', linewidth=0.5)

# Quantum state amplitudes
state_amplitudes = np.abs(quantum_state_amplitude)
ax2.bar(range(len(state_amplitudes)), state_amplitudes, color='green', alpha=0.7)
ax2.set_xlabel('Basis State Index')
ax2.set_ylabel('|Amplitude|')
ax2.set_title('Quantum State Amplitudes')

plt.tight_layout()
plt.show()

# Verify encoding
print(f"\nEncoding verification:")
print(f"Input sum of squares: {np.sum(amplitudes**2):.6f}")
print(f"Output sum of squares: {np.sum(state_amplitudes**2):.6f}")
print(f"Maximum difference: {np.max(np.abs(amplitudes - state_amplitudes)):.10f}")

### Amplitude Encoding for EO Image Patches

In [ ]:
def encode_image_patch(img, patch_size=(4, 4)):
    """
    Encode an image patch using amplitude encoding

    Args:
        img: Input image
        patch_size: Size of patch to extract

    Returns:
        Quantum state representing the patch
    """
    # Extract patch from center
    h, w = img.shape[:2]
    ph, pw = patch_size

    if len(img.shape) == 3:
        img_gray = np.mean(img, axis=2)
    else:
        img_gray = img

    # Extract center patch
    start_h, start_w = (h - ph) // 2, (w - pw) // 2
    patch = img_gray[start_h:start_h+ph, start_w:start_w+pw]

    # Flatten and normalize
    patch_flat = patch.flatten() / 255.0
    patch_norm = patch_flat / (np.linalg.norm(patch_flat) + 1e-10)

    return patch, patch_norm

# Encode patches from different land cover types
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for idx, (class_name, img) in enumerate(samples[:3]):
    # Extract and encode patch
    patch, encoded = encode_image_patch(img, patch_size=(4, 4))
    n_qubits_patch = int(np.ceil(np.log2(len(encoded))))

    # Pad to power of 2
    encoded_padded = np.zeros(2**n_qubits_patch)
    encoded_padded[:len(encoded)] = encoded
    encoded_padded = encoded_padded / np.linalg.norm(encoded_padded)

    # Show original patch
    axes[0, idx].imshow(patch, cmap='gray')
    axes[0, idx].set_title(f'{class_name}\n4×4 Patch')
    axes[0, idx].axis('off')

    # Show encoded amplitudes
    axes[1, idx].bar(range(len(encoded_padded)), encoded_padded, color='green', alpha=0.7)
    axes[1, idx].set_title(f'Quantum Amplitudes\n({n_qubits_patch} qubits)')
    axes[1, idx].set_xlabel('Basis State')
    axes[1, idx].set_ylabel('Amplitude')

plt.tight_layout()
plt.show()

# Part 3: Angle Encoding

## Theory

**Angle encoding** encodes classical data into the rotation angles of quantum gates, creating superposition states.

### Mathematical Formulation

For a classical vector $\mathbf{x} = (x_1, x_2, ..., x_n)$, angle encoding applies rotations:

$$|\psi\rangle = \bigotimes_{i=1}^{n} R(\theta_i) |0\rangle$$

where $\theta_i = f(x_i)$ and $R$ is typically $R_X$, $R_Y$, or $R_Z$:

$$R_Y(\theta) = \begin{pmatrix} \cos(\theta/2) & -\sin(\theta/2) \\ \sin(\theta/2) & \cos(\theta/2) \end{pmatrix}$$

### Properties:
- **Linear capacity**: $n$ qubits encode $n$ classical values
- **Natural superposition**: Creates entangled states
- **Scaling flexibility**: Can use $\theta = x$, $\theta = \pi x$, etc.
- **Use case**: Good for continuous features with physical meaning

### For Earth Observation:
Ideal for encoding:
- Spectral band intensities (each band → rotation angle)
- Vegetation indices (NDVI, EVI)
- Temporal features (seasonal patterns)

In [ ]:
def angle_encoding_prep(features, scaling='linear'):
    """
    Prepare angles for angle encoding

    Args:
        features: Feature vector [0, 1]
        scaling: 'linear' (θ=πx) or 'arcsin' (θ=2arcsin(√x))

    Returns:
        Angle vector
    """
    if scaling == 'linear':
        # Scale to [0, π]
        angles = np.pi * features
    elif scaling == 'arcsin':
        # Arcsin scaling for amplitude-like encoding
        angles = 2 * np.arcsin(np.sqrt(features))
    else:
        angles = features

    return angles

# Prepare angles
angles = angle_encoding_prep(features[:n_qubits], scaling='linear')

print("Angle Encoding Preparation:")
print(f"Original features: {features[:n_qubits]}")
print(f"Rotation angles:   {angles}")
print(f"Angles (degrees):  {np.degrees(angles)}")

### Implementing Angle Encoding in PennyLane

In [ ]:
dev_angle = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev_angle)
def angle_encoding_circuit(angles, rotation_gate='RY'):
    """
    Create quantum circuit with angle encoding

    Args:
        angles: Rotation angles for each qubit
        rotation_gate: 'RX', 'RY', or 'RZ'
    """
    # Apply rotation gates
    for i, angle in enumerate(angles):
        if rotation_gate == 'RX':
            qml.RX(angle, wires=i)
        elif rotation_gate == 'RY':
            qml.RY(angle, wires=i)
        elif rotation_gate == 'RZ':
            qml.RZ(angle, wires=i)

    return qml.state()

# Compare different rotation gates
rotation_types = ['RX', 'RY', 'RZ']
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, rot_type in enumerate(rotation_types):
    quantum_state = angle_encoding_circuit(angles, rotation_gate=rot_type)
    probabilities = np.abs(quantum_state) ** 2

    # Plot probabilities
    axes[idx].bar(range(len(probabilities)), probabilities, color='purple', alpha=0.7)
    axes[idx].set_xlabel('Basis State')
    axes[idx].set_ylabel('Probability')
    axes[idx].set_title(f'Angle Encoding with {rot_type} Gates')
    axes[idx].set_xlim(-1, 2**n_qubits)

plt.tight_layout()
plt.show()

### Angle Encoding for Spectral Band Analysis

In [ ]:
def extract_spectral_features(img, n_bands=3):
    """
    Extract mean spectral values from image

    Args:
        img: RGB image
        n_bands: Number of spectral bands

    Returns:
        Spectral feature vector
    """
    if len(img.shape) == 3:
        # Mean value per channel
        spectral_features = [np.mean(img[:, :, i]) / 255.0 for i in range(min(n_bands, img.shape[2]))]
    else:
        spectral_features = [np.mean(img) / 255.0]

    return np.array(spectral_features)

# Compare spectral signatures
fig, axes = plt.subplots(2, len(samples), figsize=(15, 6))

spectral_data = []
for idx, (class_name, img) in enumerate(samples):
    # Show image
    axes[0, idx].imshow(img)
    axes[0, idx].set_title(class_name)
    axes[0, idx].axis('off')

    # Extract and encode spectral features
    spectral = extract_spectral_features(img, n_bands=3)
    spectral_data.append((class_name, spectral))
    angles_spectral = angle_encoding_prep(spectral, scaling='linear')

    # Encode in quantum circuit
    dev_spectral = qml.device('default.qubit', wires=3)

    @qml.qnode(dev_spectral)
    def spectral_circuit(angles):
        for i, angle in enumerate(angles):
            qml.RY(angle, wires=i)
        return qml.probs(wires=[0, 1, 2])

    probs = spectral_circuit(angles_spectral)

    # Plot probability distribution
    axes[1, idx].bar(range(8), probs, color='teal', alpha=0.7)
    axes[1, idx].set_xlabel('Basis State')
    axes[1, idx].set_ylabel('Probability')
    axes[1, idx].set_title(f'R={spectral[0]:.2f}, G={spectral[1]:.2f}, B={spectral[2]:.2f}')

plt.tight_layout()
plt.show()